In [7]:
import sympy as sp
import numpy as np
from docplex.mp.model import Model
from quaos.hamiltonian import random_pauli_hamiltonian
from quaos.paulis.utils import find_symplectic_maps
from pyscipopt import Model as scip_model, quicksum

import numpy as np
from time import time


In [54]:
def solve_HT_equals_PH(H, d):
    n, k = H.shape  # H is n×k

    mdl = Model("HT_equals_PH")

    # Unknown matrix M ∈ ℤ_d^{k×k}
    M = [[mdl.integer_var(lb=0, ub=d - 1, name=f"M_{i}_{j}") for j in range(k)] for i in range(k)]

    # Permutation matrix P ∈ {0,1}^{n×n}
    P = [[mdl.binary_var(name=f"P_{i}_{j}") for j in range(n)] for i in range(n)]

    # Enforce P to be a permutation matrix
    for i in range(n):
        mdl.add_constraint(mdl.sum(P[i][j] for j in range(n)) == 1)  # each row: 1 entry is 1
    for j in range(n):
        mdl.add_constraint(mdl.sum(P[i][j] for i in range(n)) == 1)  # each col: 1 entry is 1

    # P ≠ I: enforce that not all diagonal entries are 1
    mdl.add_constraint(mdl.sum(P[i][i] for i in range(n)) <= n - 1)

    # Enforce H M^T ≡ P H mod d
    for i in range(n):      # row index
        for l in range(k):  # column index
            # lhs = (H M^T)[i, l] = sum_j H[i,j] * M[l,j]
            lhs = mdl.sum(int(H[i, j]) * M[l][j] for j in range(k))
            # rhs = (P H)[i, l] = sum_r P[i,r] * H[r,l]
            rhs = mdl.sum(P[i][r] * int(H[r, l]) for r in range(n))

            # Modular constraint: lhs ≡ rhs mod d ⇔ lhs - rhs = d * z
            z = mdl.integer_var(name=f"Z_{i}_{l}")
            mdl.add_constraint(lhs - rhs == d * z)

    # Solve the model
    solution = mdl.solve(log_output=True)

    if not solution:
        print("No solution found.")
        return None, None

    # Extract solution values
    M_val = np.array([[int(round(solution[M[i][j]])) % d for j in range(k)] for i in range(k)])
    P_val = np.array([[int(round(solution[P[i][j]])) for j in range(n)] for i in range(n)])

    return M_val, P_val

In [55]:
failures = []

for i in range(1):
    d = 2
    n_paulis = 10
    n_qudits = 10
    ham = random_pauli_hamiltonian(n_paulis, [d] * n_qudits, mode='uniform')
    H = ham.symplectic()

    M, P = solve_HT_equals_PH(H, d=d)

    if M is not None:
        print("Matrix M:")
        print(M)
        print("Permutation P:")
        print(P)

        # Verify: H @ M.T ≡ P @ H  (mod d)
        left = (H @ M.T) % d
        right = (P @ H) % d
        print("Verification:")
        print(np.array_equal(left, right))
        if not np.array_equal(left, right) or np.all(M - np.eye(M.shape[0], dtype=int) == 0):
            failures.append((H, M, P))

AttributeError: 'PauliSum' object has no attribute 'symplectic'

In [8]:


def solve_HT_equals_PH_scip(H, d, c=None):

    n_rows, n_cols = H.shape  # general shape
    model = scip_model("HT_equals_PH")

    # M ∈ ℤ_d^{n_cols × n_cols}  - always 2n x 2n
    M = [[model.addVar(vtype="I", lb=0, ub=d - 1, name=f"M_{i}_{j}") for j in range(n_cols)] for i in range(n_cols)]

    # P ∈ {0,1}^{n_rows × n_rows}  - depends on number of paulis only, not number of qudits
    P = [[model.addVar(vtype="B", name=f"P_{i}_{j}") for j in range(n_rows)] for i in range(n_rows)]

    # P is a permutation matrix
    for i in range(n_rows):
        model.addCons(quicksum(P[i][j] for j in range(n_rows)) == 1)
    for j in range(n_rows):
        model.addCons(quicksum(P[i][j] for i in range(n_rows)) == 1)
    model.addCons(quicksum(P[i][i] for i in range(n_rows)) <= n_rows - 1)

    # Optional group constraint
    if c is not None and np.all(c == c[0]):
        for i in range(n_rows):
            for j in range(n_rows):
                if c[i] != c[j]:
                    model.addCons(P[i][j] == 0)

    # H M^T = P H  (mod d)
    for i in range(n_rows):
        for l in range(n_cols):
            lhs = quicksum(H[i, j] * M[l][j] for j in range(n_cols))  # H[i] ⋅ M^T[:,l]
            rhs = quicksum(P[i][r] * int(H[r, l]) for r in range(n_rows))
            z = model.addVar(vtype="I", name=f"Z_{i}_{l}")
            model.addCons(lhs - rhs == d * z)

    # Solve
    model.optimize()
    if model.getStatus() != "optimal":
        print("No solution found.")
        return None, None

    M_val = np.array([[round(model.getVal(M[i][j])) % d for j in range(n_cols)] for i in range(n_cols)], dtype=int)
    P_val = np.array([[round(model.getVal(P[i][j])) for j in range(n_rows)] for i in range(n_rows)], dtype=int)
    return M_val, P_val


In [10]:
H = np.array([
    [1, 0, 1, 1],
    [0, 1, 0, 1],
    [0, 1, 0, 0],
    [1, 0, 0, 1]
])

c = np.array([1, 1, 1, 2])  # Only allow swaps within blocks 0 or 1
d = 5

M, P = solve_HT_equals_PH_scip(H, d, c)
print("M =\n", M)
print("P =\n", P)

M =
presolving:
(round 1, fast)       12 del vars, 4 del conss, 0 add conss, 20 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 8 clqs
(round 2, fast)       12 del vars, 4 del conss, 0 add conss, 20 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 8 clqs
(round 3, exhaustive) 12 del vars, 4 del conss, 0 add conss, 20 chg bounds, 0 chg sides, 0 chg coeffs, 9 upgd conss, 0 impls, 8 clqs
   (0.0s) probing cycle finished: starting next cycle
(round 4, exhaustive) 13 del vars, 4 del conss, 0 add conss, 21 chg bounds, 0 chg sides, 0 chg coeffs, 9 upgd conss, 61 impls, 36 clqs
   (0.0s) probing cycle finished: starting next cycle
(round 5, exhaustive) 14 del vars, 4 del conss, 0 add conss, 21 chg bounds, 0 chg sides, 0 chg coeffs, 9 upgd conss, 67 impls, 36 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00

In [3]:
failures = []
successes = []
failures2 = []
successes2 = []

time_solve1 = 0
print("Starting tests...")
n_tests = 1
for i in range(n_tests):
    d = 2
    n_paulis = 15
    n_qudits = 5
    ham = random_pauli_hamiltonian(n_paulis, [d] * n_qudits, mode='random')
    H = ham.symplectic_matrix()
    coeffs = ham.weights

    print('Running test', i+1, 'of', n_tests)

    time1 = time()
    M, P = solve_HT_equals_PH_scip(H, d, coeffs)
    time_solve1 += time() - time1
    if M is not None:
        left = (H @ M.T) % d
        right = (P @ H) % d
        print(np.array_equal(left, right))
        if not np.array_equal(left, right) or np.all(M - np.eye(M.shape[0], dtype=int) == 0):
            failures.append((H, M, P))
        else:
            successes.append((H, M, P))

if len(failures) == 0:
    print("All tests passed for SCIP solver.")
else:
    print(f"Failed tests for SCIP solver: {len(failures)}")
    for H, M, P in failures:
        print("H =\n", H)
        print("M =\n", M)
        print("P =\n", P)
print(f"Total time for SCIP solver: {time_solve1:.2f} seconds, average {time_solve1 / n_tests:.2f} seconds per test.")


Starting tests...


NameError: name 'random_pauli_hamiltonian' is not defined

In [57]:
def solve_HT_equals_HMt_plus_balanced_A(H, d):

    n, k = H.shape
    model = scip_model("H_Mt_minus_H_balanced")

    # M is now k x k
    M = [[model.addVar(vtype="I", lb=0, ub=d-1, name=f"M_{i}_{j}") for j in range(k)] for i in range(k)]
    A = [[model.addVar(vtype="I", lb=-d, ub=d, name=f"A_{i}_{j}") for j in range(k)] for i in range(n)]

    # Constraint: H M^T - H == A
    for i in range(n):
        for l in range(k):
            lhs = quicksum(H[i, j] * M[l][j] for j in range(k))  # (H @ M.T)[i, l]
            rhs = H[i, l] + A[i][l]
            model.addCons(lhs == rhs)

    # Column sums of A must be 0
    for j in range(k):
        model.addCons(quicksum(A[i][j] for i in range(n)) == 0)

    # Pairwise row sum constraints: for any row sum s, there must be another row with sum -s
    row_sums = [model.addVar(vtype="I", lb=-d*k, ub=d*k, name=f"row_sum_{i}") for i in range(n)]
    for i in range(n):
        model.addCons(row_sums[i] == quicksum(A[i][j] for j in range(k)))

    # Add balance constraints: sum over all row_sums == 0 (stronger than pairwise in small instances)
    model.addCons(quicksum(row_sums) == 0)

    # Objective: you can add one or set to zero
    model.setObjective(0)

    model.optimize()

    if model.getStatus() == "optimal":
        M_val = np.array([[round(model.getVal(M[i][j])) % d for j in range(k)] for i in range(k)])
        A_val = np.array([[round(model.getVal(A[i][j])) for j in range(k)] for i in range(n)])
        return M_val.T, A_val  # Return M^T to match H @ M.T
    else:
        print("No solution found.")
        return None, None

In [52]:
failures = []
successes = []
failures2 = []
successes2 = []

time_solve1 = 0
time_solve2 = 0
print("Starting tests...")
n_tests = 1
for i in range(n_tests):
    d = 2
    n_paulis = 5
    n_qudits = 5
    ham = random_pauli_hamiltonian(n_paulis, [d] * n_qudits, mode='uniform')
    H = ham.symplectic_matrix()
    coeffs = ham.weights

    print('Running test', i+1, 'of', n_tests)

    time1 = time()
    M, P = solve_HT_equals_PH_scip(H, d, coeffs)
    time_solve1 += time() - time1
    if M is not None:
        left = (H @ M.T) % d
        right = (P @ H) % d
        print(np.array_equal(left, right))
        if not np.array_equal(left, right) or np.all(M - np.eye(M.shape[0], dtype=int) == 0):
            failures.append((H, M, P))
        else:
            successes.append((H, M, P))

    time2 = time()
    M2, A2 = solve_HT_equals_HMt_plus_balanced_A(H, d)
    time_solve2 += time() - time2
    if M is not None:
        left = (H @ M2.T - H) % d
        right = (A2) % d
        print(np.array_equal(left, right))
        if not np.array_equal(left, right) or np.all(M - np.eye(M.shape[0], dtype=int) == 0):
            failures2.append((H, M2, A2))
        else:
            successes2.append((H, M2, A2))

if len(failures) == 0:
    print("All tests passed for SCIP solver.")
else:
    print(f"Failed tests for SCIP solver: {len(failures)}")
    for H, M, P in failures:
        print("H =\n", H)
        print("M =\n", M)
        print("P =\n", P)
if len(failures2) == 0:
    print("All tests passed for SCIP solver.")
else:
    print(f"Failed tests for SCIP solver: {len(failures)}")
    for H, M, P in failures:
        print("H =\n", H)
        print("M =\n", M)
        print("P =\n", P)
print(f"Total time for SCIP solver: {time_solve1:.2f} seconds, average {time_solve1 / n_tests:.2f} seconds per test.")
print(f"Total time for SCIP solver (balanced A): {time_solve2:.2f} seconds, average {time_solve2 / n_tests:.2f} seconds per test.")


Starting tests...
Running test 1 of 1
presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 50 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 34 clqs
(round 2, exhaustive) 0 del vars, 0 del conss, 0 add conss, 50 chg bounds, 0 chg sides, 0 chg coeffs, 58 upgd conss, 0 impls, 34 clqs
   (0.0s) probing: 80/145 (55.2%) - 0 fixings, 0 aggregations, 15 implications, 0 bound changes
   (0.0s) probing aborted: 50/50 successive totally useless probings
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) symmetry computation finished: 22 generators found (max: 1500, log10 of symmetry group size: 0.0) (symcode time: 0.00)
dynamic symmetry handling statistics:
   orbitopal reduction:       no components
   orbital reduction:          2 components of sizes 5, 5
   lexicographic reduction:   10 permutations with support sizes 2, 30, 2, 2, 2, 2, 2, 30, 2, 2
handled 14 out of 14 symmetry components
(round 3

TypeError: unsupported operand type(s) for %: 'pyscipopt.scip.Expr' and 'int'

In [40]:
print(f"Total time for SCIP solver: {time_solve1:.2f} seconds, average {time_solve1 / n_tests:.2f} seconds per test.")
print(f"Total time for SCIP solver (balanced A): {time_solve2:.2f} seconds, average {time_solve2 / n_tests:.2f} seconds per test.")

Total time for SCIP solver: 0.05 seconds, average 0.05 seconds per test.
Total time for SCIP solver (balanced A): 0.01 seconds, average 0.01 seconds per test.


In [48]:
for H, M, A in failures2:
    # print("H =\n", H)
    # print("M =\n", M)
    # print("A =\n", A)
    print("Verification: H @ M.T - H == A")
    print(np.array_equal((H @ M.T ) ,H+ A ))
    print((H @ M.T))
    print(H)


Verification: H @ M.T - H == A
False
[[1. 0. 3. 0. 1. 0. 0. 1. 2. 0.]
 [0. 1. 1. 0. 1. 0. 0. 0. 2. 0.]
 [0. 0. 2. 0. 2. 0. 1. 0. 1. 0.]
 [1. 1. 2. 0. 2. 0. 1. 0. 1. 0.]
 [1. 1. 4. 0. 3. 0. 0. 1. 1. 0.]]
[[1. 0. 0. 0. 0. 1. 1. 1. 1. 1.]
 [0. 1. 0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 1. 0. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 1. 0. 0. 0. 0.]
 [1. 1. 0. 1. 1. 1. 1. 1. 0. 1.]]
